# 1. Variable andf Configuration Set up

In [ ]:
import sys
!{sys.executable} -m pip install pandas==1.1.5
!{sys.executable} -m pip install numpy==1.19.1
!{sys.executable} -m pip install scikit-learn==0.22.2

In [ ]:
import joblib
import boto3
import re
import os
from config import BUCKET

# 2. Import buckets and data

In [ ]:
path = "./Data"

df_raw = pd.read_csv('{}/nb_purchase_propensity_model_raw_data.csv000'.format(path))
df_policy = pd.read_csv('{}/nb_purchase_propensity_model_raw_data_compliment_view1.csv000'.format(path))
df_quote = pd.read_csv('{}/nb_purchase_propensity_model_raw_data_compliment_view2.csv000'.format(path))

def nric_check(row):
        #     print(type(row['nric']))
        if len(str(row['nric'])) == 9 and str(row['nric'])[:1] in ['S', 'T']:
            return 'SGPR'
        elif len(str(row['nric'])) == 9 and str(row['nric'])[:1] in ['F', 'G']:
            return 'FOREIGNER'
        else:
            return 'UNKNOWN'

# Combining csv files

In [ ]:
df_raw_quote = df_raw.merge(df_quote, left_on='quote_origin_system_number', right_on='current_quote_origin_system_number', how='left', indicator=True)
df_raw_quote = df_raw_quote.rename(columns={"_merge": "matched_quote"})
df_combined = df_raw_quote.merge(df_policy, left_on='quote_origin_system_number', right_on='current_quote_origin_system_number', how='left', indicator=True)
df_combined = df_combined.rename(columns={"_merge": "matched_policy"})
df_combined = df_combined.drop(columns=['product_type_y', 'current_quote_origin_system_number_x', 'current_quote_origin_system_number_y'])
df_combined = df_combined.rename(columns={"applicant_nric_x": "nric", "applicant_nric_y": "nric_quote", "individual_id_number": "nric_policy", "product_type_x": "product_type",})
df_combined['nric_type'] = df_combined.apply(nric_check, axis=1)

In [ ]:
df_combined['applicant_dob'] = pd.to_datetime(df_combined['applicant_dob'], errors = 'coerce')
df_combined['quote_issue_date'] = df_combined['quote_issue_date'].values.astype('datetime64[ns]')
df_combined['quote_latest_update_date'] = df_combined['quote_latest_update_date'].values.astype('datetime64[ns]')
df_combined['policy_issue_date'] = pd.to_datetime(df_combined['policy_issue_date'], errors = 'coerce')
df_combined['first_quote_created_before_date'] = pd.to_datetime(df_combined['first_quote_created_before_date'], errors = 'coerce')
df_combined['latest_quote_created_before_date'] = pd.to_datetime(df_combined['latest_quote_created_before_date'], errors = 'coerce')
df_combined['scv_first_policy_issue_date'] = pd.to_datetime(df_combined['scv_first_policy_issue_date'], errors = 'coerce')
df_combined['scv_latest_policy_issue_date'] = pd.to_datetime(df_combined['scv_latest_policy_issue_date'], errors = 'coerce')
df_combined['first_interaction_date'] = pd.to_datetime(df_combined['first_interaction_date'], errors = 'coerce')
df_combined['last_interaction_date'] = pd.to_datetime(df_combined['last_interaction_date'], errors = 'coerce')

# Save dataset 

In [ ]:
df_combined.to_csv("{}/df_combined.csv".format(path))